### lodlitparser

In [1]:
import json
import requests
import time
import re
import sys
from io import BytesIO
from zipfile import ZipFile
from SPARQLWrapper import SPARQLWrapper, JSON
# install NLTK, download wordnet31 ('https://github.com/nltk/nltk_data/blob/gh-pages/packages/corpora/wordnet31.zip');
# put the content of 'wordnet31' to 'wordnet' in 'nltk_data/corpora' (there are issues with importing wordnet31 from nltk.corpus)
from nltk.corpus import wordnet as wn
# download OpenDutchWordnet from 'https://github.com/cultural-ai/OpenDutchWordnet'

### Wikidata

In [20]:
def wd(qids:list,lang:str,user_agent:str) -> dict:
    
    """
    Requesting labels, aliases, descriptions of Wikidata entities
    qids: list of entity IDs (str) (requests 50 entities at a time slicing the list)
    lang: str with language code; for example, 'en' or 'nl';
    (see language codes in Wikidata: https://www.wikidata.org/w/api.php?action=help&modules=wbgetentities)
    user_agent: str with user-agent's info; required by Wikidata (see: https://meta.wikimedia.org/wiki/User-Agent_policy)
    Returns a dict: {'QID': {'type': '',
                     'id': 'QID',
                     'labels': {'lang': {'language': 'lang', 'value': ''}},
                     'descriptions': {'lang': {'language': 'lang','value': ''}},
                     'aliases': {'lang': [{'language': 'lang', 'value': ''}
    """
    
    # 'wbgetentities' constant params
    url = "https://www.wikidata.org/w/api.php"
    params = {"action":"wbgetentities",
              "ids":"", # string of entities (max=50)
              "props":"labels|aliases|descriptions",
              "languages":lang,
              "format":"json"}
    headers = {"user-agent":user_agent}

    # - N LOOPS - #

    # if there's a remainder
    if len(qids) % 50 > 0:
        loops = len(qids) // 50 + 1 # add another loop for requests
    else:
        loops = len(qids) // 50

    # - REQUEST LOOPS - #   

    # counters to slice qids

    start_quid_str = 0
    end_quid_str = 0

    for i in range(0,loops):
        ids_string = "" # putting Qs in one string
        end_quid_str = end_quid_str + 50 # max 50 entities per request

        for q in qids[start_quid_str:end_quid_str]:
            ids_string = ids_string + f"{q}|"

        start_quid_str = start_quid_str + 50

        # updating params

        params["ids"] = ids_string.rstrip("|")

        # sending a request
        d = requests.get(url,params=params,headers=headers)
        literals = d.json() # claims per request

    return literals['entities']

In [23]:
wd(qids,lang,user_agent)

{'Q592': {'type': 'item',
  'id': 'Q592',
  'labels': {'nl': {'language': 'nl', 'value': 'gay'}},
  'descriptions': {'nl': {'language': 'nl',
    'value': 'synoniem voor homosexueel'}},
  'aliases': {'nl': [{'language': 'nl', 'value': 'homoseksueel persoon'}]}},
 'Q486680': {'type': 'item',
  'id': 'Q486680',
  'labels': {'nl': {'language': 'nl', 'value': 'travestie'}},
  'descriptions': {'nl': {'language': 'nl',
    'value': 'bewust kleding dragen van het andere geslacht, anders dan voor een acteerrol'}},
  'aliases': {'nl': [{'language': 'nl', 'value': 'travestiet'},
    {'language': 'nl', 'value': 'travesti'},
    {'language': 'nl', 'value': 'transvestitisme'}]}}}

### AAT

In [25]:
sparql = SPARQLWrapper("http://vocab.getty.edu/sparql")

In [9]:
aat_uri = ['300236748']

In [10]:
lang = 'en'

In [45]:
def aat(aat_uri:list,lang:str) -> dict:
    '''
    Querying prefLabel, altLabel, scopeNote, rdfs comments of concepts in AAT;
    Sends SPARQL queries to the AAT endpoint via SPARQLwrapper 
    aat_uri: list of AAT concepts IDs (str) ['ID']
    lang: str 'en' or 'nl'
    Returns a dict with query results: {'ID':{'lang':'en',
                                              'prefLabel':'',
                                              'altLabels':[],
                                              'scopeNote':'',
                                              'prefLabel_comment':'',
                                              'altLabel_comment':''}
    '''
    
    sparql = SPARQLWrapper("http://vocab.getty.edu/sparql")
    
    if lang == 'en':
        lang_code = '300388277'
    if lang == 'nl':
        lang_code = '300388256'
        
    result_dict = {}
        
    for uri in aat_uri:
        
        result_dict[uri] = {}
        
        query_string = '''SELECT ?prefLabel (GROUP_CONCAT(?altLabel;SEPARATOR="#") AS ?altLabels)
        ?scopeNote ?prefLabel_comment ?altLabel_comment
        WHERE {aat:''' + uri + ''' xl:prefLabel ?pL .?pL dcterms:language aat:''' + lang_code + ''';
        xl:literalForm ?prefLabel .
        OPTIONAL {?pL rdfs:comment ?prefLabel_comment . }
        OPTIONAL {aat:''' + uri + ''' xl:altLabel ?aL .
        ?aL dcterms:language aat:''' + lang_code + ''';
        xl:literalForm ?altLabel . 
        OPTIONAL { ?aL rdfs:comment ?altLabel_comment . }}
        OPTIONAL {aat:''' + uri + ''' skos:scopeNote / dcterms:language aat:'''+ lang_code + ''';
        skos:scopeNote / rdf:value ?scopeNote . }}
        GROUP BY ?prefLabel ?scopeNote ?prefLabel_comment ?altLabel_comment'''
        
        sparql.setQuery(query_string)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        
        altLabels = []
        scopeNote = None
        prefLabel_comment = None
        altLabel_comment = None
        
        for result in results['results']['bindings']:
            
            if 'altLabels' in result:
                altLabels = result['altLabels']['value'].split('#')
            if 'scopeNote' in result:
                scopeNote = result['scopeNote']['value']
            if 'prefLabel_comment' in result:
                prefLabel_comment = result['prefLabel_comment']['value']
            if 'altLabel_comment' in result:
                altLabel_comment = result['altLabel_comment']['value']

            result_dict[uri]['lang'] = lang
            result_dict[uri]['prefLabel'] = result['prefLabel']['value']
            result_dict[uri]['altLabels'] = altLabels
            result_dict[uri]['prefLabel_comment'] = prefLabel_comment
            result_dict[uri]['altLabel_comment'] = altLabel_comment
            result_dict[uri]['scopeNote'] = scopeNote
    print(query_string)    
    return result_dict

In [18]:
aat_uri_test = '300236748'
lang_code = '300388277'

In [19]:
query_string = '''SELECT ?prefLabel (GROUP_CONCAT(?altLabel;SEPARATOR="#") AS ?altLabels) ?scopeNote ?prefLabel_comment ?altLabel_comment

WHERE {

  aat:''' + aat_uri_test + ''' xl:prefLabel ?pL .
  ?pL dcterms:language aat:''' + lang_code + ''';
    xl:literalForm ?prefLabel .
  
  OPTIONAL {?pL rdfs:comment ?prefLabel_comment . }
  
  OPTIONAL {
  aat:''' + aat_uri_test + ''' xl:altLabel ?aL .
  ?aL dcterms:language aat:''' + lang_code + ''';
    xl:literalForm ?altLabel . 
    OPTIONAL { ?aL rdfs:comment ?altLabel_comment . }
  }
  
  OPTIONAL {aat:''' + aat_uri_test + ''' skos:scopeNote / dcterms:language aat:'''+ lang_code + ''';
                skos:scopeNote / rdf:value ?scopeNote . }
}
GROUP BY ?prefLabel ?scopeNote ?prefLabel_comment ?altLabel_comment'''

In [37]:
#### Debugging AAT

In [38]:
aat_uri_test = '300236748'
lang = 'en'

In [39]:
sparql = SPARQLWrapper("http://vocab.getty.edu/sparql")

if lang == 'en':
    lang_code = '300388277'
if lang == 'nl':
    lang_code = '300388256'

result_dict = {}

for uri in aat_uri:

    result_dict[uri] = {}

    query_string = '''SELECT ?prefLabel (GROUP_CONCAT(?altLabel;SEPARATOR="#") AS ?altLabels)
    ?scopeNote ?prefLabel_comment ?altLabel_comment
    WHERE {aat:''' + uri + ''' xl:prefLabel ?pL .?pL dcterms:language aat:''' + lang_code + ''';
    xl:literalForm ?prefLabel .
    OPTIONAL {?pL rdfs:comment ?prefLabel_comment . }
    OPTIONAL {aat:''' + uri + ''' xl:altLabel ?aL .
    ?aL dcterms:language aat:''' + lang_code + ''';
    xl:literalForm ?altLabel . 
    OPTIONAL { ?aL rdfs:comment ?altLabel_comment . }}
    OPTIONAL {aat:''' + uri + ''' skos:scopeNote / dcterms:language aat:'''+ lang_code + ''';
    skos:scopeNote / rdf:value ?scopeNote . }}
    GROUP BY ?prefLabel ?scopeNote ?prefLabel_comment ?altLabel_comment'''

    sparql.setQuery(query_string)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    altLabels = []
    scopeNote = None
    prefLabel_comment = None
    altLabel_comment = None

    for result in results['results']['bindings']:

        if 'altLabels' in result:
            altLabels = result['altLabels']['value'].split('#')
        if 'scopeNote' in result:
            scopeNote = result['scopeNote']['value']
        if 'prefLabel_comment' in result:
            prefLabel_comment = result['prefLabel_comment']['value']
        if 'altLabel_comment' in result:
            altLabel_comment = result['altLabel_comment']['value']

        result_dict[uri]['lang'] = lang
        result_dict[uri]['prefLabel'] = result['prefLabel']['value']
        result_dict[uri]['altLabels'] = altLabels
        result_dict[uri]['prefLabel_comment'] = prefLabel_comment
        result_dict[uri]['altLabel_comment'] = altLabel_comment
        result_dict[uri]['scopeNote'] = scopeNote

In [42]:
if 'scopeNote' in results:
    print(True)

In [44]:
results['results']['bindings']

[{'prefLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'dwarfs'},
  'altLabels': {'type': 'literal', 'value': "dwarf#dwarf's#dwarfs'#dwarves"},
  'scopeNote': {'xml:lang': 'en',
   'type': 'literal',
   'value': 'People who are abnormally small in stature.'}}]

In [46]:
aat(['300236748'],lang)

SELECT ?prefLabel (GROUP_CONCAT(?altLabel;SEPARATOR="#") AS ?altLabels)
        ?scopeNote ?prefLabel_comment ?altLabel_comment
        WHERE {aat:300236748 xl:prefLabel ?pL .?pL dcterms:language aat:300388277;
        xl:literalForm ?prefLabel .
        OPTIONAL {?pL rdfs:comment ?prefLabel_comment . }
        OPTIONAL {aat:300236748 xl:altLabel ?aL .
        ?aL dcterms:language aat:300388277;
        xl:literalForm ?altLabel . 
        OPTIONAL { ?aL rdfs:comment ?altLabel_comment . }}
        OPTIONAL {aat:300236748 skos:scopeNote / dcterms:language aat:300388277;
        skos:scopeNote / rdf:value ?scopeNote . }}
        GROUP BY ?prefLabel ?scopeNote ?prefLabel_comment ?altLabel_comment


{'300236748': {'lang': 'en',
  'prefLabel': 'dwarfs',
  'altLabels': ['dwarf', "dwarf's", "dwarfs'", 'dwarves'],
  'prefLabel_comment': None,
  'altLabel_comment': None,
  'scopeNote': 'People who are abnormally small in stature.'}}

In [72]:
# draft

results = {'head': {'vars': ['prefLabel',
   'altLabels',
   'scopeNote',
   'prefLabel_comment',
   'altLabel_comment']},
 'results': {'bindings': [{'prefLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'dwarfs'},
    'altLabels': {'type': 'literal', 'value': "dwarf#dwarf's#dwarfs'#dwarves"},
    'scopeNote': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'People who are abnormally small in stature.'}}]}}

In [80]:
lang = 'en'
uri = '300236748'
altLabels = []
scopeNote = None
prefLabel_comment = None
altLabel_comment = None

for result in results['results']['bindings']:
    
    result_dict = {uri:{}}
    
    if 'altLabels' in result:
        altLabels = result['altLabels']['value'].split('#')
    if 'scopeNote' in results:
        scopeNote = result['scopeNote']['value']
    if 'prefLabel_comment' in results:
        prefLabel_comment = result['prefLabel_comment']['value']
    if 'altLabel_comment' in results:
        altLabel_comment = result['altLabel_comment']['value']
        
    result_dict[uri]['lang'] = lang
    result_dict[uri]['prefLabel'] = result['prefLabel']['value']
    result_dict[uri]['altLabels'] = altLabels
    result_dict[uri]['prefLabel_comment'] = prefLabel_comment
    result_dict[uri]['altLabel_comment'] = altLabel_comment
    result_dict[uri]['scopeNote'] = scopeNote
        
    print(result_dict)

{'300236748': {'lang': 'en', 'prefLabel': 'dwarfs', 'altLabels': ['dwarf', "dwarf's", "dwarfs'", 'dwarves'], 'prefLabel_comment': None, 'altLabel_comment': None, 'scopeNote': None}}


In [ ]:
{'300236748':{'lang':'en',
              'prefLabel':'',
              'altLabels':[],
              'scopeNote':'',
              'prefLabel_comment':'',
              'altLabel_comment':''}

### NMVW

In [1]:
# parsing the json file created before: '/Users/anesterov/WM/nmvw_thesaurus.json'

In [19]:
term_ids = ['10076668','10060821','003']

In [5]:
nmvw_path = '/Users/anesterov/WM/nmvw_thesaurus.json'

In [22]:
def nmvw(term_ids:list) -> dict:
    '''
    Getting info about terms by their handle IDs in NMVW-thesaurus
    term_ids: list of term IDs (str)
    Returns a dict with query results: {'ID': {'prefLabel': '',
                                               'altLabel': [],
                                               'notes': [],
                                               'exactMatch': '',
                                               'scheme': ''}}
    '''
    
    results_nmvw = {}
    
    for term_id in term_ids:
        handle = 'https://hdl.handle.net/20.500.11840/termmaster' + term_id
        results_nmvw[handle] = nmvw_json.get(handle)
        
    return results_nmvw

In [23]:
nmvw(term_ids,nmvw_path)

{'https://hdl.handle.net/20.500.11840/termmaster10076668': {'prefLabel': 'Aboriginal Australiërs',
  'altLabel': ['Aborigines',
   'Aboriginal',
   'Australisch Aboriginal',
   'Aboriginals',
   'Australian Aboriginal'],
  'notes': ['Algemene aanduiding voor de oorspronkelijke bewoners van Australië',
   "Price & AAT-Ned; Words that Metter  ; in AAT: DESC='Australisch Aboriginal' ; v16: AAT-ID & BE toegevoegd ; v19: DESC gewijzigd, was 'Aboriginal'"],
  'exactMatch': 'http://vocab.getty.edu/aat/300021862',
  'scheme': 'Cultuur'},
 'https://hdl.handle.net/20.500.11840/termmaster10060821': {'prefLabel': 'Nagoya',
  'altLabel': [],
  'notes': ['Times Atlas 1994 & TGN ; WMR (2010) ; v01: tech.val.'],
  'exactMatch': 'https://www.geonames.org/1856057',
  'scheme': 'Geografie'},
 'https://hdl.handle.net/20.500.11840/termmaster003': None}

### Princeton WordNet

In [13]:
def pwn(synsets:list) -> dict:
    '''
    Getting lemmata, definition, examples of a synset
    synsets: a list of synsets IDs (str)
    Return a dict: {'synset_id': {'lemmata': '',
                                   'definition': '',
                                   'examples': []}
    Requires NLTK, wordnet corpus version 3.1
    '''
    
    results_pwn = {}
    
    for s in synsets:
        synset = wn.synset(s)
        lemmata = [l.name() for l in synset.lemmas()]
        definition = synset.definition()
        examples = synset.examples()
        
        # writing results 
        results_pwn[s] = {'lemmata': lemmata,
                         'definition': definition,
                         'examples': examples}
        
    return results_pwn

In [14]:
pwn(['savage.n.01'])

{'savage.n.01': {'lemmata': ['savage', 'barbarian'],
  'definition': 'a member of an uncivilized people',
  'examples': []}}

### ODWN

In [2]:
def odwn(le_ids:list, path_odwn:str) -> dict:
    '''
    Getting Lemma, sense definitions, examples, synset ID, synset definitions of ODWN Lexical Entries
    le_ids: list of Lexical Entries IDs (str)
    path_odwn: str path to the directory with OpenDutchWordnet (not including the module itself, for example 'user/Downloads')
    Returns a dict: {'le_id': {'lemma': '',
                                'sense_def': '',
                                'examples': [],
                                'synset_ID': '',
                                'synset_def': []}
    '''
    # importing ODWN
    sys.path.insert(0,path_odwn)
    from OpenDutchWordnet import Wn_grid_parser
    # creating an instance
    instance = Wn_grid_parser(Wn_grid_parser.odwn)
    
    # importing all synset definitions
    path_to_glosses = "https://raw.githubusercontent.com/cultural-ai/wordsmatter/main/ODWN/odwn_synset_glosses.json"
    synset_glosses = requests.get(path_to_glosses).json()
    
    results_odwn = {}

    for le_id in le_ids:
        le = instance.les_find_le(le_id)
        synset_id = le.get_synset_id()

        sense_def = le.get_definition()
        if sense_def == '':
            sense_def = None

        results_odwn[le_id] = {'lemma': le.get_lemma(),
                                'sense_def': sense_def,
                                'examples': le.get_sense_example(),
                                'synset_ID': synset_id,
                                'synset_def': synset_glosses.get(synset_id)}
    return results_odwn

In [5]:
odwn(le_ids,path_odwn)

{'koelie-n-1': {'lemma': 'koelie',
  'sense_def': 'bediende zoals in de tropen',
  'examples': [],
  'synset_ID': 'odwn-10-101160806-n',
  'synset_def': ['bediende zoals in de tropen']},
 'koelie-n-2': {'lemma': 'koelie',
  'sense_def': None,
  'examples': [],
  'synset_ID': 'eng-30-10035809-n',
  'synset_def': ['iemand die werkt tegen betaling']},
 'Jap-n-1': {'lemma': 'Jap',
  'sense_def': 'Japanner',
  'examples': [],
  'synset_ID': 'odwn-10-103201483-n',
  'synset_def': ['Japanner']},
 'homoseksueel-n-1': {'lemma': 'homoseksueel',
  'sense_def': 'homofiel persoon',
  'examples': [],
  'synset_ID': 'eng-30-10182913-n',
  'synset_def': ['homoseksueel persoon',
   'homoseksueel persoon',
   'homofiel persoon']},
 'mohammedaan-n-1': {'lemma': 'mohammedaan',
  'sense_def': 'aanhanger v.d. islam',
  'examples': [],
  'synset_ID': 'odwn-10-107525787-n',
  'synset_def': ['aanhanger v.d. islam']},
 'moor-n-1': {'lemma': 'moor',
  'sense_def': 'iemand met een donkere huidskleur',
  'examples

In [4]:
path_odwn = "/Users/anesterov/reps"

In [47]:
sys.path.insert(0,path_odwn)

In [48]:
from OpenDutchWordnet import Wn_grid_parser

In [49]:
instance = Wn_grid_parser(Wn_grid_parser.odwn)

In [72]:
# importing all synset definitions
path_to_glosses = "https://raw.githubusercontent.com/cultural-ai/wordsmatter/main/ODWN/odwn_synset_glosses.json"
synset_glosses = requests.get(path_to_glosses).json()

In [3]:
le_ids = ['koelie-n-1','koelie-n-2','Jap-n-1','homoseksueel-n-1','mohammedaan-n-1','moor-n-1','Moor-n-2']

In [80]:
results_odwn = {}

for le_id in le_ids:
    le = instance.les_find_le(le_id)
    synset_id = le.get_synset_id()
    
    sense_def = le.get_definition()
    if sense_def == '':
        sense_def = None
        
    
    results_odwn[le_id] = {'lemma': le.get_lemma(),
                            'sense_def': sense_def,
                            'examples': le.get_sense_example(),
                            'synset_ID': synset_id,
                            'synset_def': synset_glosses.get(synset_id)}

In [81]:
results_odwn

{'koelie-n-1': {'lemma': 'koelie',
  'sense_def': 'bediende zoals in de tropen',
  'examples': [],
  'synset_ID': 'odwn-10-101160806-n',
  'synset_def': ['bediende zoals in de tropen']},
 'koelie-n-2': {'lemma': 'koelie',
  'sense_def': None,
  'examples': [],
  'synset_ID': 'eng-30-10035809-n',
  'synset_def': ['iemand die werkt tegen betaling']},
 'Jap-n-1': {'lemma': 'Jap',
  'sense_def': 'Japanner',
  'examples': [],
  'synset_ID': 'odwn-10-103201483-n',
  'synset_def': ['Japanner']},
 'homoseksueel-n-1': {'lemma': 'homoseksueel',
  'sense_def': 'homofiel persoon',
  'examples': [],
  'synset_ID': 'eng-30-10182913-n',
  'synset_def': ['homoseksueel persoon',
   'homoseksueel persoon',
   'homofiel persoon']},
 'mohammedaan-n-1': {'lemma': 'mohammedaan',
  'sense_def': 'aanhanger v.d. islam',
  'examples': [],
  'synset_ID': 'odwn-10-107525787-n',
  'synset_def': ['aanhanger v.d. islam']},
 'moor-n-1': {'lemma': 'moor',
  'sense_def': 'iemand met een donkere huidskleur',
  'examples

In [62]:
le = instance.les_find_le('neger-n-1')

In [60]:
le

In [69]:
instance.les_lemma_synonyms('kaffer')

{'barbaar',
 'boer',
 'boerenhufter',
 'boerenkaffer',
 'boerenkarhengst',
 'boerenkinkel',
 'boerenlul',
 'boerenpummel',
 'botterik',
 'grobbejanus',
 'heikneuter',
 'hork',
 'hufter',
 'kaffer',
 'kinkel',
 'lomperd',
 'lomperik',
 'palurk',
 'plebejer',
 'plomperd',
 'primitieveling',
 'proleet',
 'pummel',
 'raudauwer',
 'rouwdouw',
 'rouwdouwer',
 'vlegel',
 'woudezel'}

#### check where a query term is found per resource:
* Wikidata: labels, aliases
* AAT: prefLabel, altLabel
* NMVW: prefLabel, altLabel
* Princeton WordNet: no checking, only lemmata
* ODWN: no checking, only lemma

In [1]:
from LODlitParser import lodlitparser

In [35]:
lodlitparser.pwn(['white.n.01', 'white.a.02', 'white.s.05'])

{'white.n.01': {'lemmata': ['White', 'White_person', 'Caucasian'],
  'definition': 'a Caucasian',
  'examples': []},
 'white.a.02': {'lemmata': ['white'],
  'definition': 'of or belonging to a racial group having light skin coloration',
  'examples': ['voting patterns within the white population']},
 'white.s.05': {'lemmata': ['white', 'lily-white'],
  'definition': 'restricted to whites only',
  'examples': ['under segregation there were even white restrooms and white drinking fountains',
   'a lily-white movement which would expel Negroes from the organization']}}

In [3]:
lodlitparser.nmvw(['10076668'])

{'https://hdl.handle.net/20.500.11840/termmaster10076668': {'prefLabel': 'Aboriginal Australiërs',
  'altLabel': ['Aborigines',
   'Aboriginal',
   'Australisch Aboriginal',
   'Aboriginals',
   'Australian Aboriginal'],
  'notes': ['Algemene aanduiding voor de oorspronkelijke bewoners van Australië',
   "Price & AAT-Ned; Words that Metter  ; in AAT: DESC='Australisch Aboriginal' ; v16: AAT-ID & BE toegevoegd ; v19: DESC gewijzigd, was 'Aboriginal'"],
  'exactMatch': 'http://vocab.getty.edu/aat/300021862',
  'scheme': 'Cultuur'}}

In [4]:
lodlitparser.wd(['Q592'],'en','test_query')

{'Q592': {'type': 'item',
  'id': 'Q592',
  'labels': {'en': {'language': 'en', 'value': 'gay'}},
  'descriptions': {'en': {'language': 'en',
    'value': 'term referring to a homosexual person or the trait of homosexuality'}},
  'aliases': {'en': [{'language': 'en', 'value': 'homosexual person'},
    {'language': 'en', 'value': 'homosexuals'},
    {'language': 'en', 'value': 'homosexual people'},
    {'language': 'en', 'value': 'gay people'},
    {'language': 'en', 'value': 'gay person'},
    {'language': 'en', 'value': 'ghey'},
    {'language': 'en', 'value': 'ゲイ'}]}}}

In [6]:
lodlitparser.aat(['300016430','300049946'],'nl')

{'300016430': {'lang': 'nl',
  'prefLabel': 'Pygmee',
  'altLabels': [''],
  'prefLabel_comment': None,
  'altLabel_comment': None,
  'scopeNote': 'Te gebruiken om werken te beschrijven van bepaalde Afrikaanse volkeren die leven in Kameroen, Equatoriaal-Guinea, de Democratische Republiek Congo, Congo, de Centraal-Afrikaanse Republiek, Rwanda en Burundi. De term Pygmee heeft soms een negatieve bijklank. Waar mogelijk verdient het gebruik van de specifieke naam van een subgroep de voorkeur.'},
 '300049946': {'lang': 'nl',
  'prefLabel': 'stammen',
  'altLabels': ['stam', 'volksstammen'],
  'prefLabel_comment': None,
  'altLabel_comment': None,
  'scopeNote': 'Sociale groepen waarvan de leden geacht worden af te stammen van een gemeenschappelijke voorouder, en die bestaan uit talrijke families, geslachten of dorpen. Meestal bewonen ze een specifiek geografisch gebied, zijn ze homogeen op cultureel, religieus en taalkundig gebied en zijn ze politiek verenigd door één leider of hoofdman.'}}